In [22]:
from pulp import LpProblem, LpVariable, LpMaximize, LpMinimize, LpStatus, lpSum, value, apis

# from pulp import pulpTestAll
# pulpTestAll()

# 1. Initialise model
model = LpProblem("Maximize LP", LpMaximize)

# 2. Define Decision Variables: 
x1 = LpVariable('X1', lowBound=0, upBound=None, cat="Continuous")
x2 = LpVariable('X2', lowBound=0, upBound=None, cat="Continuous")
x3 = LpVariable('X3', lowBound=0, upBound=None, cat="Continuous")
x4 = LpVariable('X4', lowBound=0, upBound=None, cat="Continuous")

# 3. Define objective function: 
model += lpSum(2 * x1 + 4 * x2 + 3 * x3 + x4), 'obj'

# 4. Define constraints: our model's limitations
model += lpSum(3 * x1 + x2 +x3 + 4 * x4) <= 12, 'c1'
model += lpSum(x1 -3*x2 + 2*x3 + 3 * x4) <= 7, 'c2'  
model += lpSum(2*x1 + x2 + 3*x3 - x4) <= 10, 'c3'
        

# 5. Solve model
model.solve(apis.PULP_CBC_CMD(msg=0))

print(f"Status: {LpStatus[model.status]}\n")
for name, c in model.constraints.items():
    print(f"{name}: slack = {c.slack:.2f}, shadow price = {c.pi:.2f}")

for v in model.variables():
    print(v.name,"=", v.varValue)
print("Objective = ", value(model.objective))


Status: Optimal

c1: slack = -0.00, shadow price = 1.00
c2: slack = 37.00, shadow price = -0.00
c3: slack = -0.00, shadow price = 3.00
X1 = 0.0
X2 = 10.4
X3 = 0.0
X4 = 0.4
Objective =  42.0


## Change of an objective function coefficient

1. Variable is non-basic (it takes a value of zero in the solution). For example, let us modify the

In [23]:
model.constraints['c1'][x1]
oldvalue = model.objective[x1]
model.objective[x1] = 10
model.solve()
print("X1 = {} ".format(x1.varValue))
print("X2 = {} ".format(x2.varValue))
print("X3 = {} ".format(x3.varValue))
print("X4 = {} ".format(x4.varValue))
print(f"Optimal value of the function: {(x1.varValue * 2) + (x2.varValue * 4) + (x3.varValue * 3) + (x4.varValue)}")
model.objective[x1] = oldvalue  # return to original value

X1 = 2.0 
X2 = 6.0 
X3 = 0.0 
X4 = 0.0 
Optimal value of the function: 28.0


## Changing the RHS of a constraint
It depends if it is binding or not binding

In [24]:

model.constraints['c1'] = lpSum(3 * x1 + x2 +x3 + 4 * x4) <= 13
model.solve()
print("X1 = {} ".format(x1.varValue))
print("X2 = {} ".format(x2.varValue))
print("X3 = {} ".format(x3.varValue))
print("X4 = {} ".format(x4.varValue))
print(f"Optimal value of the function: {(x1.varValue * 2) + (x2.varValue * 4) + (x3.varValue * 3) + (x4.varValue)}")

X1 = 0.0 
X2 = 10.6 
X3 = 0.0 
X4 = 0.6 
Optimal value of the function: 43.0


## Adding a constrain

1. Original solution satisfies new constraint:
* If your initial solution already satisfies the new constraint, the problem is still feasible, and you might not need to change anything.
* In this case, you can still use your original solution because it meets the new requirements. The value of the objective function (whether it is a cost to minimize or profit to maximize) will not increase. If your original solution was already optimal, it remains optimal.
2. Original solution does not satisfy new constraint:
* If the original solution no longer satisfies the new constraint, the problem might become infeasible, or you may need to find a new solution that satisfies all constraints.
* If the problem is still feasible (i.e., a solution exists), you must find a new solution. However, since the new constraint makes the problem harder (i.e., more restrictive), the new solution will likely have a worse objective value than the original one. The value of the objective function will decrease, because adding constraints typically limits the feasible solution space, making it harder to achieve a better outcome.